In [1]:
!pip install rasterio
!pip install earthpy
!pip install rioxarray
!pip install geopandas
!!pip install pyshp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 12.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 7.5 MB/s 
     |████████████████████████████████| 1.0 MB 9.0 MB/s 
     |████████████████████████████████| 6.3 MB 10.6 MB/s 
     |████████████████████████████████| 16.7 MB 197 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 2.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for rioxarray: filename=rioxarray-0.9.1-py3-none-any.whl size=54611 sha256=93d10880222b443a56451bcecadbc9e6133e95907118c52e4f74e748866689e4
  Stored in directory: /root/.cache/pip/wheels/07/da/9e/1cc57b2e7a29a206893d

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting pyshp',
 '  Downloading pyshp-2.3.0-py2.py3-none-any.whl (46 kB)',
 '\x1b[?25l',
 '\x1b[K     |███████                         | 10 kB 20.9 MB/s eta 0:00:01',
 '\x1b[K     |██████████████▏                 | 20 kB 27.8 MB/s eta 0:00:01',
 '\x1b[K     |█████████████████████▎          | 30 kB 18.4 MB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████▍   | 40 kB 7.4 MB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████████| 46 kB 2.6 MB/s ',
 '\x1b[?25hInstalling collected packages: pyshp',
 'Successfully installed pyshp-2.3.0']

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/EarthEngineToGeoTIFF_v3.py EarthEngineToGeoTIFF_v3.py

In [4]:
import ee
from osgeo import gdal
import rasterio
from rasterio.plot import show as showRasterio
import geopandas as gpd
import shapefile
from shapely.geometry import MultiPoint, Point, Polygon

import pandas as pd
import numpy as np
from PIL import Image
import os
import datetime
import time
from datetime import datetime, timedelta
import random
import matplotlib.pyplot as plt

# my script
from EarthEngineToGeoTIFF_v3 import getSentinalS2SRImage

import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import rioxarray as rxr

random.seed(1234)

In [5]:
# Download file to google drive
!unzip /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/cb_2018_us_state_20m.zip

Archive:  /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/cb_2018_us_state_20m.zip
  inflating: cb_2018_us_state_20m.shp.ea.iso.xml  
  inflating: cb_2018_us_state_20m.shp.iso.xml  
  inflating: cb_2018_us_state_20m.shp  
  inflating: cb_2018_us_state_20m.shx  
  inflating: cb_2018_us_state_20m.dbf  
  inflating: cb_2018_us_state_20m.prj  
 extracting: cb_2018_us_state_20m.cpg  


In [6]:
ee.Authenticate()

ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=zA8S5qP5eoOeAlQ-tLgM2NDvL5Pidn0mfSPnN0JY79I&tc=ys4ggrqQcgu9rKow6xQ0IRTCOqVxXLXb3X_JuBzraPw&cc=JHD49dvCsRN0qcBplYQ8_Mqikhg5NbV1qZ7PBmoxeLs

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AdQt8qjv9jhzlkEPXm-YEJsJWHkA5ot9F7_pRDjzD2yQkZERF3wU3THYmW0

Successfully saved authorization token.


In [7]:
def imgQuality(dirIM,listIM):
  """
  Check image quality for pixels
  Arguments:
  dirIm (str): directory where images are located
  listIM(list): list of images in directory
  Returns:
  df (DataFrame): returns a dataframe {'name':images, 'cloudQuality':cloudQuality,'missingQuality':Quality}
  """
  df = pd.DataFrame()
  images = []
  cloudQuality = []
  Quality = []
  for im in listIM:
    imd = Image.open(dirIM+im, 'r')
    imA = np.asarray(imd)
    if imA.mean() == 0:
      quality = 0
      cloudquality = 0
      Quality.append(quality)
      cloudQuality.append(cloudquality)
      images.append(im)      
    else:
      vals = imA.shape[0] * imA.shape[1] * imA.shape[2] # denominator
      #quality = 1- (np.bincount(np.reshape(imA, imA.size))[0] / vals) 
      quality = 1 - (np.count_nonzero(imA ==0 )/vals) # number missing pixels/ total number of pixels
      cloudquality = 1- (np.count_nonzero(imA == 1) / vals) # number of white pixels (cloud) / total number pixels
      Quality.append(quality)
      cloudQuality.append(cloudquality)
      images.append(im)
  df = df.append(pd.DataFrame({'name':images, 'cloudQuality':cloudQuality,'missingQuality':Quality}))
  return df



In [8]:
#!rm -r jpegs006at64

In [9]:
#! cp -r  /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/jpegs006at64 jpegs006at64

In [10]:
# dir006 = "jpegs006at64/"
# l006 = os.listdir(dir006)
# len(l006)

# Create list of Sinkhole IDs with picture
# tileID = []
# for im in l006:
#     tileID.append(int(im.split("-")[0]))
# len(tileID)

In [11]:
#sh1 = pd.read_excel("/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/Florida_Subsidence_Incident_Reports.xlsx", sheet_name = 0)
sh1 = pd.read_excel("/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/sinkhole_10_years.xlsx")
sh1.head()

,X,Y,OBJECTID,REF_NUM,DATE_REV,EVENT_DATE,Year,TRUE_SINK,LONGDD,LATDD,...,LIMVIS,CAVVIS,SUBRATE,PROPDAM,REPAIR_S,DRAINSTR,SOILTYPE,COMMENTS,COMMENTS_2,ACCESS_
0,-82.682258,28.360988,67,14-758,2015/11/03 00:00:00+00,2012/06/20 00:00:00+00,2012,U,-82.682255,28.360983,...,U,U,R,Y,Unknown,Unknown,UNKNOWN,"40' dia, 20' deep; sinkhole destroyed back hal...",NaN,NaN
1,-84.865330,29.744150,106,49-001,2015/10/30 00:00:00+00,2012/08/25 00:00:00+00,2012,U,-84.865326,29.744144,...,N,U,R,Y,Repaired,Unknown,UNKNOWN,SWO# 2012-6049: subsidence occurred in the roa...,Location confirmed via Google Maps image on in...,NaN
2,-82.991917,30.288370,124,37-435,2014/12/08 00:00:00+00,2012/06/27 00:00:00+00,2012,U,-82.991913,30.288364,...,U,U,U,U,Unknown,Unknown,NaN,NaN,Report 37-436 says 4 sinkholes located near ch...,NaN
3,-81.945712,27.946343,126,16-896,2021/07/30 00:00:00+00,2014/02/14 00:00:00+00,2014,U,-81.945708,27.946338,...,N,U,U,U,Repaired,Unknown,NaN,Sinkhole was caused by digging near a retentio...,NaN,NaN
4,-81.745600,28.787240,144,11-559,2021/08/05 00:00:00+00,2015/08/24 00:00:00+00,2015,U,-81.745597,28.787234,...,U,U,U,U,Unknown,Unknown,NaN,Lake County Dispatch reports a 4 x 4 sinkhole ...,Report #: 2015-6132,NaN


In [12]:
#dfselect = sh1[(sh1["OBJECTID"].isin(tileID))]
dfselect =sh1.copy()
dfselect.head()

,X,Y,OBJECTID,REF_NUM,DATE_REV,EVENT_DATE,Year,TRUE_SINK,LONGDD,LATDD,...,LIMVIS,CAVVIS,SUBRATE,PROPDAM,REPAIR_S,DRAINSTR,SOILTYPE,COMMENTS,COMMENTS_2,ACCESS_
0,-82.682258,28.360988,67,14-758,2015/11/03 00:00:00+00,2012/06/20 00:00:00+00,2012,U,-82.682255,28.360983,...,U,U,R,Y,Unknown,Unknown,UNKNOWN,"40' dia, 20' deep; sinkhole destroyed back hal...",NaN,NaN
1,-84.865330,29.744150,106,49-001,2015/10/30 00:00:00+00,2012/08/25 00:00:00+00,2012,U,-84.865326,29.744144,...,N,U,R,Y,Repaired,Unknown,UNKNOWN,SWO# 2012-6049: subsidence occurred in the roa...,Location confirmed via Google Maps image on in...,NaN
2,-82.991917,30.288370,124,37-435,2014/12/08 00:00:00+00,2012/06/27 00:00:00+00,2012,U,-82.991913,30.288364,...,U,U,U,U,Unknown,Unknown,NaN,NaN,Report 37-436 says 4 sinkholes located near ch...,NaN
3,-81.945712,27.946343,126,16-896,2021/07/30 00:00:00+00,2014/02/14 00:00:00+00,2014,U,-81.945708,27.946338,...,N,U,U,U,Repaired,Unknown,NaN,Sinkhole was caused by digging near a retentio...,NaN,NaN
4,-81.745600,28.787240,144,11-559,2021/08/05 00:00:00+00,2015/08/24 00:00:00+00,2015,U,-81.745597,28.787234,...,U,U,U,U,Unknown,Unknown,NaN,Lake County Dispatch reports a 4 x 4 sinkhole ...,Report #: 2015-6132,NaN


In [13]:
tiles = pd.read_excel("/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/sentinel_tiles.xlsx")
tilesFL = tiles[(tiles["State"]=="FL")]
tilesFL.head()

,State,Row,TILE_ID,MULTIPOLIGON,Comments
0,FL,1,16RDV,MULTIPOLYGON(((-88.0548969859722 31.6311891453...,Shared between Alabama and Florida. Mostly Con...
1,FL,1,16REV,MULTIPOLYGON(((-87.0002109102755 31.6355471130...,Shared between Alabama and Florida. Mostly Con...
2,FL,1,16RFV,MULTIPOLYGON(((-85.9455247432055 31.6311926291...,Shared between Alabama and Florida. Mostly Con...
7,FL,2,16RDU,MULTIPOLYGON(((-88.0449388796329 30.7288635166...,Half is ocean - half into Alabama
8,FL,2,16REU,MULTIPOLYGON(((-87.0002089188631 30.7330700317...,half is ocean


In [14]:
def get_polygon(tiles, tile_id):
    """
    Get coordinates of the polygon

    Parameters:
    tiles(DataFrame): Dataframe with tiles information
    tile_id(str): tile id from sentinel-2

    Returns:
    coords(tuple): polygon coordinates in tuple

    """
    mp = tiles[(tiles["TILE_ID"]==tile_id)][["MULTIPOLIGON"]].values
    mp1 = mp[0][0][15:]
    mp1 = mp1[0:len(mp1)-3]
    mp1 = mp1.split(",")
    
    coords = []
    for e1 in mp1:
        sp1 = e1.split(" ")
        coords.append([float(sp1[0]), float(sp1[1])])
    
    return (coords)

In [15]:
def getRandomTile(rdtiles):
  """
  Returns a random Sentinel 2 tile 

  Parameters:
  rdtiles(DataFrame): Dataframe with tiles information

  Returns:
  tilename(str): name of random tile
  """
  tilesnames = rdtiles["TILE_ID"].unique()
  tilename = random.choice(tilesnames)
  return tilename

In [16]:
def get_us_border_polygon():
    """
    Creates a dictionary containing the Polygons of each US state
    """
    sf = shapefile.Reader("/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/cb_2018_us_state_20m.shp")
    shapes = sf.shapes()
    fields = sf.fields
    records = sf.records()
    state_polygons = {}
    for i, record in enumerate(records):
        state = record[5]
        points = shapes[i].points
        poly = Polygon(points)
        state_polygons[state] = poly

    return state_polygons

#us border
state_polygons = get_us_border_polygon() 
#check if in one of the states then True, else False
def in_us(lat, lon):
    """
    Checks to see if a given coordinates is within a state
    Parameters:
    lat(float): latitude coordinate
    lon(float): longitude coordinate

    Returns:
    state(str): returns the state name
    """
    p = Point(lon, lat)
    for state, poly in state_polygons.items():
        if poly.contains(p):
            return state
    return None
# Reference: https://stackoverflow.com/questions/42501349/given-latitude-longitude-say-if-the-coordinate-is-within-continental-us-or-not

In [17]:
def getRandomcoordinates(rdtiles, num=92):
  """
  Generates random coordinates within the state of Florida
  Parameters:
  rdtiles(DataFrame): DataFrame of sentinel2 tiles
  num(int): number of tiles to generate

  Returns:
  valid_coordinates(list): returns the list of coordinates within the state
  """
  valid_coordinates = []
  i = 0
  while i <num:
    coords = getRandomLatLon(rdtiles)
    # print(coords)
    state = in_us(coords[0],coords[1])
    #print(state)
    if state == 'Florida':
      valid_coordinates.append(coords)
      i+=1
  print('Created {} coordinates ...'.format(len(valid_coordinates)))
  return valid_coordinates
      #print(state)
    
# get backup points

In [18]:
def getRandomLatLon(rdtiles):
  """
  Get a random random coordinate within a tile
  Parameters:
  rdtiles(DataFrame): DataFrame of sentinel2 tiles

  Returns:
  lat,lon(tuple): returns coordinates in format (lat,lon)

  """
    
  tilesnames = rdtiles["TILE_ID"].unique()
    
  tilename = random.choice(tilesnames)
  coords = np.array(get_polygon(rdtiles, tilename))
    
  lonmin = coords[:,0].min()
  lonmax = coords[:,0].max()
  latmin = coords[:,1].min()
  latmax = coords[:,1].max()
        
  lon = random.uniform(lonmin, lonmax)
  lat = random.uniform(latmin, latmax)
    
  return lat, lon

In [19]:
def convertdate(date, formatD):
  """
  Convert date from string format to specified date format
  Parameters:
  date(str): date string
  formatD(str): format for date

  Returns:
  datetime(date): formatted datetime
  """
    
  if type(date) == str:
    d = date[0:10]
    return(datetime.strptime(d,"%Y/%m/%d"))
  else:
    return("NaN")

In [20]:
dfselect

,X,Y,OBJECTID,REF_NUM,DATE_REV,EVENT_DATE,Year,TRUE_SINK,LONGDD,LATDD,...,LIMVIS,CAVVIS,SUBRATE,PROPDAM,REPAIR_S,DRAINSTR,SOILTYPE,COMMENTS,COMMENTS_2,ACCESS_
0,-82.682258,28.360988,67,14-758,2015/11/03 00:00:00+00,2012/06/20 00:00:00+00,2012,U,-82.682255,28.360983,...,U,U,R,Y,Unknown,Unknown,UNKNOWN,"40' dia, 20' deep; sinkhole destroyed back hal...",NaN,NaN
1,-84.865330,29.744150,106,49-001,2015/10/30 00:00:00+00,2012/08/25 00:00:00+00,2012,U,-84.865326,29.744144,...,N,U,R,Y,Repaired,Unknown,UNKNOWN,SWO# 2012-6049: subsidence occurred in the roa...,Location confirmed via Google Maps image on in...,NaN
2,-82.991917,30.288370,124,37-435,2014/12/08 00:00:00+00,2012/06/27 00:00:00+00,2012,U,-82.991913,30.288364,...,U,U,U,U,Unknown,Unknown,NaN,NaN,Report 37-436 says 4 sinkholes located near ch...,NaN
3,-81.945712,27.946343,126,16-896,2021/07/30 00:00:00+00,2014/02/14 00:00:00+00,2014,U,-81.945708,27.946338,...,N,U,U,U,Repaired,Unknown,NaN,Sinkhole was caused by digging near a retentio...,NaN,NaN
4,-81.745600,28.787240,144,11-559,2021/08/05 00:00:00+00,2015/08/24 00:00:00+00,2015,U,-81.745597,28.787234,...,U,U,U,U,Unknown,Unknown,NaN,Lake County Dispatch reports a 4 x 4 sinkhole ...,Report #: 2015-6132,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,-81.222853,28.631983,4127,77-719,NaN,2018/07/14 00:00:00+00,2018,U,-81.222850,28.631978,...,Y,U,R,Y,Planned,Unknown,NaN,sinkhole reported in road of Wembley Place nea...,SWO# 2018-4337,NaN
851,-83.089320,30.297916,4134,37-417,2016/02/29 00:00:00+00,2012/06/29 00:00:00+00,2012,U,-83.089316,30.297910,...,U,U,U,U,Unknown,Unknown,NaN,NaN,"Cannot locate exact address, did not move",NaN
852,-81.810803,28.530095,4137,11-557,NaN,2014/10/05 00:00:00+00,2014,U,-81.810800,28.530090,...,U,U,U,N,Unknown,Unknown,NaN,The sinkhole is currently about 10 feet wide a...,NaN,NaN
853,-84.234614,30.299485,4138,55-751,NaN,2014/12/24 00:00:00+00,2014,U,-84.234610,30.299479,...,N,N,R,N,Unknown,Unknown,NaN,Sinkholes formed overnight after a heavy rainf...,NaN,NaN


In [21]:
dfselect["DateD"] = dfselect.apply(lambda row: convertdate(row["EVENT_DATE"], "%Y-%m-%d"), axis=1)
dfselect = dfselect[dfselect['DateD']>='2019-11-01']

In [22]:
dfselect

,X,Y,OBJECTID,REF_NUM,DATE_REV,EVENT_DATE,Year,TRUE_SINK,LONGDD,LATDD,...,CAVVIS,SUBRATE,PROPDAM,REPAIR_S,DRAINSTR,SOILTYPE,COMMENTS,COMMENTS_2,ACCESS_,DateD
83,-81.932475,28.944928,428,11-581,2019/12/17 00:00:00+00,2019/12/11 00:00:00+00,2019,N,-81.932472,28.944923,...,U,U,Y,Unknown,Unknown,NaN,Residence 20 ft away is threatened,SWO# 2019-6915,NaN,2019-12-11
84,-82.161323,29.208174,429,36-797,2020/01/06 00:00:00+00,2020/01/05 00:00:00+00,2020,N,-82.161320,29.208169,...,U,U,U,Unknown,Unknown,NaN,"1,000,000 gallons of treated water from the re...",SWO# 2020-87,NaN,2020-01-05
86,-82.625674,30.192630,431,29-507,2019/12/05 00:00:00+00,2019/12/05 00:00:00+00,2019,N,-82.625670,30.192624,...,U,U,Y,Unknown,Unknown,NaN,"10 Families impacted, no major roadways closed",SWO# 2019-6813,NaN,2019-12-05
89,-86.093305,30.682650,434,60-503,2021/07/14 00:00:00+00,2020/02/18 00:00:00+00,2020,U,30.682749,-86.093285,...,U,U,U,Unknown,Unknown,NaN,SWP# 2020-1034,NaN,NaN,2020-02-18
90,-86.093300,30.682644,435,55-763,2021/07/12 00:00:00+00,2020/05/31 00:00:00+00,2020,U,-84.232400,30.299860,...,N,U,N,Unknown,Unknown,NaN,pool backwash was the source of creation.,NaN,10072 Green Fountain Rd,2020-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,-82.487170,28.552020,3932,08-787,2021/08/13 00:00:00+00,2021/08/11 00:00:00+00,2021,U,-82.487170,28.552020,...,N,R,U,Unknown,None,NaN,Mr. Bifulco advised the subsidence is located ...,NaN,Brookridge Blvd,2021-08-11
807,-82.559860,28.783541,3933,02-895,2021/10/05 00:00:00+00,2021/10/04 00:00:00+00,2021,U,-82.560020,28.783400,...,N,R,Y,Unknown,Unknown,NaN,all NB lanes of S Suncoast Blvd are closed.,NaN,NB lanes of S Suncoast Blvd @ Oakridge Dr,2021-10-04
808,-81.751759,28.583100,3934,11-585,2021/10/15 00:00:00+00,2021/10/14 00:00:00+00,2021,U,-81.751757,28.583100,...,N,U,N,Planned,Unknown,NaN,Coordinates innacurate. Placed point in accord...,Road closed on 14th and back open after 3 hrs ...,NaN,2021-10-14
809,-81.884420,28.755252,3935,11-586,2021/11/10 00:00:00+00,2021/11/06 00:00:00+00,2021,U,-81.884420,28.755260,...,U,R,N,Unknown,Unknown,NaN,Lake County EM advises that there is a potenti...,This period of time the state recieved continu...,"CR 33 to Diamond Blvd. Turn right on Rudy Ave,...",2021-11-06


In [23]:
def create_dir(tfdir):
    """
    Creates a new directory if it does not exist
  
    Parameters:
    tfdir (str): location and name of new directory

    """
    if os.path.exists(tfdir)==False:
        os.makedirs(tfdir)
        print('{} directory created'.format(tfdir))
    else:
        print('{} directory already exists'.format(tfdir))

In [24]:
#create_dir('random006')
create_dir('random006_10yr')
#create_dir("image006")
create_dir("image006_10yr")

random006_10yr directory created
image006_10yr directory created


In [25]:
def get_random_images(sh_target, sze, folder, tilesFL, num=3, td_1=365, td_2=60):
    """
    Get satellite images for randomly generated coordinates

    Parameters:
      sh_target(DataFrame): dataframe of sinkhole incidents
      sze(float):size of the edge of the box in degrees
      folder(str): folder where tif files are to be saved
      tilesFL(DataFrame): dataframe of sentinel tiles
      num(int): number of images to generate
    Returns:
      Data(DataFrame): returns a dataframe with the following columns 'name', 'ID', 'lon', 'lat', 'start_date'
    """

    td = timedelta(td_1)
    td2 = timedelta(td_2)
    
    data = pd.DataFrame()
    
    fnames, labels, sh_IDs, lons, lats, sds, imgnums = [], [], [], [], [], [], []
    label = 0

    coords = getRandomcoordinates(tilesFL, num=round(sh_target.shape[0],0))
    i = 0 
    for index, row in sh_target.iterrows():
      lat, lon = coords[i][0], coords[i][1]
      i+=1
      print(lat,lon)
        
      sh_ID = row["OBJECTID"]
      dateD = row["DateD"]
      dateStr = row["EVENT_DATE"][0:10]
      startDate = dateD - td
      endDate = startDate + td2

      start_date = startDate.strftime("%Y-%m-%d")
      end_date = endDate.strftime("%Y-%m-%d")
        
      print(start_date, end_date)

        
      for z in list(range(num+1)):        
          fname = str(sh_ID)+"-R-"+start_date+"-"+end_date+"-"+str(sze*1000)+str(num)
          fnames.append(fname+'-'+str(z)+'.tif')
          sh_IDs.append(sh_ID)
          lons.append(lon)
          lats.append(lat)
          sds.append(startDate)
          imgnums.append(z)
          labels.append(label)

      _ = getSentinalS2SRImage(lon, lat, sze, folder+fname, start_date, 
                                end_date, j=num)
      time.sleep(15)
        
    data = data.append(pd.DataFrame({"name": fnames, 
                              'imgnum': imgnums,
                              'label' : labels,
                              "ID": sh_IDs, 
                              "lon": lons, 
                              "lat":lats, 
                              "start_date": sds}))
    
    return data

In [26]:
def get_satellite_images(sh_target, sze, folder, num=1, td_1=365, td_2=60):
    td = timedelta(td_1)
    td2 = timedelta(td_2)
    data = pd.DataFrame()
    label =1
    fnames, labels, sh_IDs, lons, lats, sds, imgnums = [], [], [], [], [], [], []
    for index, row in sh_target.iterrows():
        lat = row["Y"]
        lon = row["X"]
        sh_ID = row["OBJECTID"]
        dateD = row["DateD"]
        dateStr = row["EVENT_DATE"][0:10]
        startDate = dateD - td
        endDate = startDate + td2

        start_date = startDate.strftime("%Y-%m-%d")
        end_date = endDate.strftime("%Y-%m-%d")
        
        print(start_date, end_date)
        
        for z in list(range(num+1)):
          fname = str(sh_ID)+"-"+start_date+"-"+end_date+"-"+str(sze*1000)+str(num)
          fnames.append(fname+'-'+str(z)+'.tif')
          sh_IDs.append(sh_ID)
          lons.append(lon)
          lats.append(lat)
          sds.append(startDate)
          imgnums.append(z)
          labels.append(label)
        _ = getSentinalS2SRImage(lon, lat, sze, folder+fname, start_date, 
                                end_date, j=num)
        time.sleep(15)
        
    data = data.append(pd.DataFrame({"name": fnames, 
                              'imgnum': imgnums,
                              'label' : labels,
                              "ID": sh_IDs, 
                              "lon": lons, 
                              "lat":lats, 
                              "start_date": sds}))
    return data
        

In [27]:
randomIMGs =  get_random_images(dfselect, 0.006, "random006_10yr/", tilesFL, num=3, td_1=1, td_2=60)

Created 113 coordinates ...
24.98501253604042 -82.34757433239237
2019-12-10 2020-02-08
3
Creating 3-band GeoTIFF image ... 
Actual filename is random006_10yr/428-R-2019-12-10-2020-02-08-6.03-0.tif
Creating 3-band GeoTIFF image ... 
Actual filename is random006_10yr/428-R-2019-12-10-2020-02-08-6.03-1.tif
Creating 3-band GeoTIFF image ... 
Actual filename is random006_10yr/428-R-2019-12-10-2020-02-08-6.03-2.tif
Creating 3-band GeoTIFF image ... 
Actual filename is random006_10yr/428-R-2019-12-10-2020-02-08-6.03-3.tif
30.982728955320255 -85.84145421156289
2020-01-04 2020-03-04
3
Creating 3-band GeoTIFF image ... 
Actual filename is random006_10yr/429-R-2020-01-04-2020-03-04-6.03-0.tif
Creating 3-band GeoTIFF image ... 
Actual filename is random006_10yr/429-R-2020-01-04-2020-03-04-6.03-1.tif
Creating 3-band GeoTIFF image ... 
Actual filename is random006_10yr/429-R-2020-01-04-2020-03-04-6.03-2.tif
Creating 3-band GeoTIFF image ... 
Actual filename is random006_10yr/429-R-2020-01-04-2020-03

In [28]:
IMGs=get_satellite_images(dfselect, 0.006, "image006_10yr/", num=2, td_1=1, td_2=60)

2019-12-10 2020-02-08
2
Creating 3-band GeoTIFF image ... 
Actual filename is image006_10yr/428-2019-12-10-2020-02-08-6.02-0.tif
Creating 3-band GeoTIFF image ... 
Actual filename is image006_10yr/428-2019-12-10-2020-02-08-6.02-1.tif
Creating 3-band GeoTIFF image ... 
Actual filename is image006_10yr/428-2019-12-10-2020-02-08-6.02-2.tif
2020-01-04 2020-03-04
2
Creating 3-band GeoTIFF image ... 
Actual filename is image006_10yr/429-2020-01-04-2020-03-04-6.02-0.tif
Creating 3-band GeoTIFF image ... 
Actual filename is image006_10yr/429-2020-01-04-2020-03-04-6.02-1.tif
Creating 3-band GeoTIFF image ... 
Actual filename is image006_10yr/429-2020-01-04-2020-03-04-6.02-2.tif
2019-12-04 2020-02-02
2
Creating 3-band GeoTIFF image ... 
Actual filename is image006_10yr/431-2019-12-04-2020-02-02-6.02-0.tif
Creating 3-band GeoTIFF image ... 
Actual filename is image006_10yr/431-2019-12-04-2020-02-02-6.02-1.tif
Creating 3-band GeoTIFF image ... 
Actual filename is image006_10yr/431-2019-12-04-2020-

In [29]:
dirRand = "random006_10yr/"
lrand = os.listdir(dirRand)
len(lrand)

452

In [30]:
def getcornercoordinates(path):
  """
  Get coordinates for the four corners of a tif file
  
  Parameters:
  path(str): path where tif files are located
  Returns:
  df (DataFrame): with name and tuple of coordinates

  """
  names = []
  polygons = []
  files = os.listdir(path)
  for name in files:
    names.append(name)
    ds = gdal.Open(path+name)
    width = ds.RasterXSize
    height = ds.RasterYSize
    gt = ds.GetGeoTransform()
    minx = gt[0]
    miny = gt[3] + width*gt[4] + height*gt[5] 
    maxx = gt[0] + width*gt[1] + height*gt[2]
    maxy = gt[3]

    corner1 = (minx,miny)
    corner2 = (minx,maxy)
    corner3 = (maxx,maxy)
    corner4 = (maxx,miny)
    polygons.append([corner1,corner2, corner3, corner4])
  return pd.DataFrame({'name':names, 'polygon':polygons})
    

In [31]:
def convert2jpeg(tifdir, jpegdir):
    """
    Saves tiff file as jpeg
  
    Parameters:
    tifdir (str): path of tif files
    jpegdir (str): path to save jpeg files
  
    """
    
    filelist = os.listdir(tifdir)
    
    options_list = [
        '-ot Byte',
        '-of JPEG',
        '-b 3',
        '-b 2',
        '-b 1',
        '-scale'
    ]           

    options_string = " ".join(options_list)

    for filetif in filelist:
        
        jpegname = filetif.replace(".tif", ".jpeg")
                
        gdal.Translate(
            jpegdir+jpegname,
            tifdir+filetif,
            options=options_string
        )

In [32]:
def create_dir(tfdir):
    """
    Creates a new directory if it does not exist
  
    Parameters:
    tfdir (str): location and name of new directory

    """
    if os.path.exists(tfdir)==False:
        os.makedirs(tfdir)
        print('{} directory created'.format(tfdir))
    else:
        print('{} directory already exists'.format(tfdir))


In [33]:
def remove_xmls(jpegdir):
    """
    Deletes xml files in directory
  
    Parameters:
    jpegdir (str): path to delete xml files associated with jpeg files
  
    """
    for file in os.listdir(jpegdir):
        if ".xml" in file:
            os.remove(jpegdir+file)
    print('xml files deleted')

In [34]:
def crop_image(path, dimy=64, dimx=64):
    """
    Crops image to a specified dimension (i.e. 64x64 pixels)
    
    Parameters:
    path (str): path to a specific jpeg file
  
    """
    from PIL import Image 
    img = Image.open(path) 

    left = 0
    top = 0
    right = dimx
    bottom = dimy
 
    img_res = img.crop((left, top, right, bottom)) 
    img_res.save(path)

In [35]:
def crop_img_loop():
    """
    Loops through all jpeg files and crops them
    """
    for img in os.listdir(jpegdir):
        crop_image(jpegdir+img)
    print('Images were cropped to 64 by 64 pixels')


In [36]:
def selectQualityImgs(jpegdir, randomIMGs, threshold = .90, random=True):
  """
  Return DataFrame with the highest quality images identified based on a specified quality criteria

  Parameters:
    jpegdir(str): specifies directory where jpeg images are stored
    randomIMGs(DataFrame): DataFrame generated from the get_random_images function
    threshold(float): specify a number between 0 to 1 for the upper bound threshold for what is acceptable quality

  Returns:
    randomIMGs_cleaned(DataFrame)

  """
  # get list of jpeg imgs in the directory
  l006 = os.listdir(jpegdir)
  check = imgQuality(jpegdir, l006)
  # identify images that do not meet quality threshold
  poor_quality_imgs = check[check['missingQuality']<threshold]
  # get rows that have good quality
  clean_images = check[~check['name'].isin(list(poor_quality_imgs.name))].reset_index(drop=True)
  # get columns for identifying the first image for each sinkhole incident (OBJECTID and imgnum)
  clean_images['OBJECTID'] = clean_images.apply(lambda row: int(row['name'].split('-')[0]), axis=1)
  if random:
    clean_images['imgnum'] = clean_images.apply(lambda row: int(row['name'].split('-')[9].replace('.jpeg','')), axis=1)
  else: 
    clean_images['imgnum'] = clean_images.apply(lambda row: int(row['name'].split('-')[8].replace('.jpeg','')), axis=1)
  # find the first image based on cloud coverage from query
  clean_imgs = clean_images.groupby(by=['OBJECTID']).min('imgnum').reset_index()
  clean_imgs = clean_imgs[['OBJECTID','imgnum']]

  #identify these rows with first images
  match = pd.merge(clean_images,clean_imgs, on=['OBJECTID', 'imgnum'], how='inner')
  # get list of them
  names_clean = list(match.name)
  # rename to join with csv DataFrame
  names_clean = [i.replace('.jpeg','.tif') for i in names_clean]
  randomIMGs_cleaned = randomIMGs[randomIMGs['name'].isin(names_clean)].reset_index(drop=True)
  return randomIMGs_cleaned


In [37]:
dirsplit = "random006_10yr/"
create_dir('jpegsrandom006at64_10yr')
%rm /content/jpegsrandom006at64_10yr/*
jpegdir = "jpegsrandom006at64_10yr/"

convert2jpeg(dirsplit, jpegdir)
remove_xmls(jpegdir)
crop_img_loop()
randomIMGs_cleaned = selectQualityImgs(jpegdir, randomIMGs, threshold = .88)

jpegsrandom006at64_10yr directory created
rm: cannot remove '/content/jpegsrandom006at64_10yr/*': No such file or directory
xml files deleted
Images were cropped to 64 by 64 pixels


In [38]:
dirsplit = "image006_10yr/"
create_dir('jpegs006at64_10yr')
%rm /content/jpegs006at64/*

jpegdir = "jpegs006at64_10yr/"

convert2jpeg(dirsplit, jpegdir)
remove_xmls(jpegdir)
crop_img_loop()
IMGs_cleaned = selectQualityImgs(jpegdir, IMGs, threshold = .88,random=False)

jpegs006at64_10yr directory created
rm: cannot remove '/content/jpegs006at64/*': No such file or directory
xml files deleted
Images were cropped to 64 by 64 pixels


In [39]:
cleaned_all = randomIMGs_cleaned.append(IMGs_cleaned)

In [40]:
%cp -av "/content/jpegsrandom006at64_10yr" "/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script"
#%cp -av "/content/jpegs006at64_10yr" "/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/"

'/content/jpegsrandom006at64_10yr/545-R-2021-07-22-2021-09-20-6.03-2.jpeg' -> '/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/jpegsrandom006at64_10yr/545-R-2021-07-22-2021-09-20-6.03-2.jpeg'
'/content/jpegsrandom006at64_10yr/541-R-2020-04-30-2020-06-29-6.03-1.jpeg' -> '/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/jpegsrandom006at64_10yr/541-R-2020-04-30-2020-06-29-6.03-1.jpeg'
'/content/jpegsrandom006at64_10yr/2459-R-2021-09-03-2021-11-02-6.03-1.jpeg' -> '/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/jpegsrandom006at64_10yr/2459-R-2021-09-03-2021-11-02-6.03-1.jpeg'
'/content/jpegsrandom006at64_10yr/3929-R-2021-07-15-2021-09-13-6.03-3.jpeg' -> '/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/jpegsrandom006at64_10yr/3929-R-2021-07-15-2021-09-13-6.03-3.jpeg'
'/content/jpegsrandom006at64_10yr/3294-R-2022-03-02-2022-05-01-6.03-2.jp

In [41]:
#!cp /content/random_images1.csv /content/drive/MyDrive/w210

In [42]:
#randomIMGs_cleaned.to_csv("randomIMGs_cleaned.csv", index=None)

In [43]:
#!cp /content/randomIMGs_cleaned.csv /content/drive/MyDrive/w210

In [44]:
coords = getcornercoordinates('image006_10yr/')
randomcoords = getcornercoordinates('random006_10yr/')

In [45]:
coords['geometry'] = coords.apply(lambda row: Polygon(row['polygon']), axis=1)

In [46]:
randomcoords['geometry'] = randomcoords.apply(lambda row: Polygon(row['polygon']), axis=1)

In [47]:
randomcoords = randomcoords[['name','geometry']]
coords = coords[['name','geometry']]

In [48]:
randomIMGs_final = pd.merge(randomIMGs_cleaned, randomcoords, on='name', how='left')

In [49]:
IMGs_final = pd.merge(IMGs_cleaned, coords, on='name', how='left')

In [50]:
final_shapefile = IMGs_final.append(randomIMGs_final)
gdf = gpd.GeoDataFrame(final_shapefile, geometry='geometry')

gdf = gdf[['name', 'ID', 'imgnum','lon', 'lat', 'geometry']]
gdf.to_file("/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/subtiles_final_10yr_1.shp")

In [51]:
#!cp subtiles_final.shp  /content/drive/MyDrive/w210/subtiles_final.shp

In [52]:
#!cp /content/drive/MyDrive/w210/subtiles_final.shp subtiles_final.shp

In [53]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import random
import shutil
import time
import warnings
import gc 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.models as models

import cv2
from matplotlib import pyplot as plt

def make_predictions(preddir, path = '/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/model_best.pth.tar'):
    """
    Creates a DataFrame and Shapefile with the landuse factors extracted

    Parameters:
    preddir(str): path to the directory of images to make land use predictions on
    path(str): path to torch model weights artifacts (model_best.pth.tar file)
    """
    SEED=1
    random.seed(SEED)
    torch.manual_seed(SEED)
    cudnn.deterministic = True
    #########################
    # Define epochs and printing frequency
    START_EPOCH = 0
    #EPOCHS = 10
    PRINT_FREQ = 2500

    #Dynamically set batch size and workers
    avail_gpus = min(1, torch.cuda.device_count())
    batch_size = 100 if avail_gpus else 64
    #batch_size = 50
    TRAIN_BATCH= batch_size
    VAL_BATCH = batch_size
    WORKERS= int(os.cpu_count()-2/2)

    # Files locations of training and validation data
    # root_dir = "data"
    # TRAINDIR= os.path.join(root_dir,'train')
    # VALDIR= os.path.join(root_dir,'val')


    # check if cuda is available in this cell
    # if it is not available, you should not go forward!
    device = "cuda:0" if torch.cuda.is_available() else 'cpu'

    # enable algorithm optimization
    cudnn.benchmark = True

    # This cell is tagged `parameters` for papermill
    LR = 0.1
    MOMENTUM = 0.9
    WEIGHT_DECAY = 5e-4
    EPOCHS = 120

    IMG_SIZE = 224
    IMG_SIZE2 = int(round(1.15 * IMG_SIZE,0))
    imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
    imagenet_std_RGB = [0.229, 0.224, 0.225]

    IDX_CLASS_LABELS = {
    0: 'AnnualCrop',
    1: 'Forest', 
    2: 'HerbaceousVegetation',
    3: 'Highway',
    4: 'Industrial',
    5: 'Pasture',
    6: 'PermanentCrop',
    7: 'Residential',
    8: 'River',
    9: 'SeaLake'
    }
#### Load in RESNET50 model architecture and weights from best performing model
    model = torchvision.models.resnet50()

    # Freeze all layers except for the final layer
    for param in model.parameters():
        param.requires_grad = False
        
        classes = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture','PermanentCrop','Residential','River', 'SeaLake']

        PATH = path

        # Getting the final layer to match the number of classes
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, len(classes))


        optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM)

        checkpoint = torch.load(PATH,map_location='cpu')
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        epoch = checkpoint['epoch']

        model.cpu()

        model.eval()

    # Create a folder called `predictions` in the sinkhole folder, 
    # in the `predictions` folder create a `unlabeled` subfolder, 
    # paste all images that need predictions in this folder.
    PREDDIR = preddir

    transform_val = transforms.Compose([
        transforms.Resize(IMG_SIZE2),
        transforms.CenterCrop(IMG_SIZE),
    #     transforms.Grayscale(3),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB),
    ])

    pred_dataset = ImageFolder(root=PREDDIR,transform=transform_val)

    pred_loader = torch.utils.data.DataLoader(pred_dataset, batch_size=batch_size,
                                            shuffle=False, num_workers=WORKERS) 
    with torch.no_grad():
        end = time.time()
        outputs = pd.DataFrame()
        predictions = pd.DataFrame()
        for i, (images, target) in enumerate(pred_loader):
            #print(images.shape)
            #print(i)
            ### send the images and target to cuda
            images = images.to(device)
    
            # compute output
            # output = model ??? images?
            output = model(images)
            _ , prediction = torch.max(output.cpu().detach(), dim=1)
            prob = F.softmax(output, dim=1)
            predictions = predictions.append(pd.DataFrame(prediction.cpu().numpy()))
            outputs = outputs.append(pd.DataFrame(prob.cpu().numpy()))
    # Create DataFrame & GeoDataframe
    outputs.columns = classes
    outputs.reset_index(drop=True, inplace=True)
    names = pd.DataFrame({'name':os.listdir(PREDDIR+'/unlabeled')})
    final_outputs = pd.merge(names,outputs, left_index=True, right_index=True, how='inner')
    predictions.columns = ['prediction']
    predictions.reset_index(drop=True, inplace=True)
    predictions['prediction_name'] = predictions.apply(lambda row: IDX_CLASS_LABELS[row['prediction']], axis=1)
    return  pd.merge(final_outputs,predictions, left_index=True, right_index=True, how='inner')

In [54]:
def prep_predictions():
  os.mkdir('predictions')
  os.mkdir('predictions/unlabeled')

In [55]:
prep_predictions()
! cp -r /content/jpegs006at64_10yr/* predictions/unlabeled/
! cp -r /content/jpegsrandom006at64_10yr/* predictions/unlabeled/
print('{} files in predictions/unlabeled directory'.format(len(os.listdir('predictions/unlabeled'))))

791 files in predictions/unlabeled directory


In [56]:
#os.mkdir('predictions/unlabeled')

In [57]:
! cp -r /content/jpegs006at64_10yr_365_2 /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/
! cp -r /content/jpegsrandom006at64_10yr_365_2 /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/

cp: cannot stat '/content/jpegs006at64_10yr_365_2': No such file or directory
cp: cannot stat '/content/jpegsrandom006at64_10yr_365_2': No such file or directory


In [58]:
#len(os.listdir('predictions/unlabeled'))

In [85]:
!zip -r /content/jpegs006at64_10yr_1.zip /content/jpegs006at64_10yr_1

  adding: content/jpegs006at64_10yr_1/ (stored 0%)
  adding: content/jpegs006at64_10yr_1/1082-2020-10-19-2020-12-18-6.02-1.jpeg (deflated 8%)
  adding: content/jpegs006at64_10yr_1/2348-2021-09-29-2021-11-28-6.02-2.jpeg (deflated 7%)
  adding: content/jpegs006at64_10yr_1/556-2021-09-22-2021-11-21-6.02-1.jpeg (deflated 7%)
  adding: content/jpegs006at64_10yr_1/3287-2020-06-03-2020-08-02-6.02-2.jpeg (deflated 6%)
  adding: content/jpegs006at64_10yr_1/2350-2021-11-11-2022-01-10-6.02-2.jpeg (deflated 6%)
  adding: content/jpegs006at64_10yr_1/2347-2021-08-15-2021-10-14-6.02-1.jpeg (deflated 7%)
  adding: content/jpegs006at64_10yr_1/2998-2020-05-15-2020-07-14-6.02-1.jpeg (deflated 6%)
  adding: content/jpegs006at64_10yr_1/551-2021-08-30-2021-10-29-6.02-1.jpeg (deflated 6%)
  adding: content/jpegs006at64_10yr_1/2008-2020-06-06-2020-08-05-6.02-2.jpeg (deflated 9%)
  adding: content/jpegs006at64_10yr_1/2462-2021-11-30-2022-01-29-6.02-0.jpeg (deflated 8%)
  adding: content/jpegs006at64_10yr_1/206

In [86]:
!zip -r /content/jpegsrandom006at64_10yr_1.zip /content/jpegsrandom006at64_10yr_1

  adding: content/jpegsrandom006at64_10yr_1/ (stored 0%)
  adding: content/jpegsrandom006at64_10yr_1/558-R-2022-03-20-2022-05-19-6.03-0.jpeg (deflated 7%)
  adding: content/jpegsrandom006at64_10yr_1/2732-R-2021-12-23-2022-02-21-6.03-2.jpeg (deflated 7%)
  adding: content/jpegsrandom006at64_10yr_1/2347-R-2021-08-15-2021-10-14-6.03-2.jpeg (deflated 13%)
  adding: content/jpegsrandom006at64_10yr_1/3005-R-2022-03-15-2022-05-14-6.03-0.jpeg (deflated 6%)
  adding: content/jpegsrandom006at64_10yr_1/3004-R-2021-10-25-2021-12-24-6.03-2.jpeg (deflated 7%)
  adding: content/jpegsrandom006at64_10yr_1/2735-R-2022-03-15-2022-05-14-6.03-0.jpeg (deflated 8%)
  adding: content/jpegsrandom006at64_10yr_1/1077-R-2021-04-20-2021-06-19-6.03-2.jpeg (deflated 9%)
  adding: content/jpegsrandom006at64_10yr_1/551-R-2021-08-30-2021-10-29-6.03-2.jpeg (deflated 10%)
  adding: content/jpegsrandom006at64_10yr_1/3003-R-2021-09-18-2021-11-17-6.03-1.jpeg (deflated 10%)
  adding: content/jpegsrandom006at64_10yr_1/2350-R-

In [61]:
def get_predictions(name='predictions'):
  df = make_predictions(preddir='predictions')
  df.to_csv('predictions.csv',index=None)
  df['name'] = df.apply(lambda row: row['name'].replace('.jpeg','.tif'), axis = 1)
  final_shapefile2 = final_shapefile.reset_index(drop=True)
  final_shapefile2['name'] = final_shapefile2.apply(lambda row: row['name'].replace('.jpeg','.tif'), axis = 1)
  final_shapefile2['imgnum'] = final_shapefile2.apply(lambda row: int(row['name'].split('-')[-1].replace(".tif","")),axis=1)
  df['imgnum'] = df.apply(lambda row: int(row['name'].split('-')[-1].replace(".tif","")),axis=1)
  shapefile3 = pd.merge(final_shapefile2,df, on =['name','imgnum'],  how='inner')
  shapefile3.to_csv('/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/'+name+'.csv', index=None)
  gdf2 = gpd.GeoDataFrame(shapefile3, geometry='geometry')
  gdf2['start_date'] = gdf2.apply(lambda row: row['start_date'].strftime("%Y/%m/%d"),axis=1)
  gdf2.to_file("/content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/"+name+".shp")
  return gdf2, df





In [62]:
gdf2, df = get_predictions(name='predictions_10yr_group1_1')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  del sys.path[0]


In [63]:
df

,name,AnnualCrop,Forest,HerbaceousVegetation,Highway,Industrial,Pasture,PermanentCrop,Residential,River,SeaLake,prediction,prediction_name,imgnum
0,1082-2020-10-19-2020-12-18-6.02-1.tif,1.313343e-16,2.838180e-04,4.408744e-08,5.008692e-04,9.872841e-01,2.200774e-12,4.001908e-11,8.069928e-03,3.694312e-03,1.670730e-04,4,Industrial,1
1,2348-2021-09-29-2021-11-28-6.02-2.tif,2.274169e-15,4.698176e-05,1.482220e-04,6.446164e-05,7.573038e-01,1.075252e-11,2.385961e-11,2.217353e-05,2.403684e-01,2.045907e-03,4,Industrial,2
2,558-R-2022-03-20-2022-05-19-6.03-0.tif,5.981783e-17,1.770654e-07,2.340178e-09,1.694231e-05,9.995352e-01,1.013848e-10,1.526488e-09,4.868012e-08,4.455248e-04,2.116058e-06,4,Industrial,0
3,2732-R-2021-12-23-2022-02-21-6.03-2.tif,5.021705e-30,4.205182e-22,1.148874e-22,1.057557e-12,1.000000e+00,2.867687e-21,4.448067e-23,1.041213e-08,3.937533e-14,2.143715e-20,4,Industrial,2
4,2347-R-2021-08-15-2021-10-14-6.03-2.tif,1.556020e-24,6.285389e-15,6.293930e-15,2.895553e-08,9.999975e-01,3.720859e-15,4.249452e-15,2.293665e-06,1.739796e-07,8.653858e-13,4,Industrial,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,2737-2022-03-28-2022-05-27-6.02-0.tif,1.881614e-09,1.216938e-02,6.318500e-04,9.743893e-06,1.518973e-08,6.213490e-14,1.865554e-09,2.572356e-13,9.868587e-01,3.302993e-04,8,River,0
787,2066-2021-04-14-2021-06-13-6.02-2.tif,4.928116e-19,2.074677e-06,5.148085e-09,3.082241e-09,9.993284e-01,3.535076e-19,4.469499e-14,6.695478e-04,9.244077e-08,2.160292e-09,4,Industrial,2
788,2342-R-2020-04-30-2020-06-29-6.03-2.tif,6.058778e-26,1.261011e-13,5.190030e-16,5.973677e-12,1.000000e+00,6.380386e-17,1.566852e-18,1.729915e-08,1.150883e-11,6.406579e-17,4,Industrial,2
789,2009-2021-07-06-2021-09-04-6.02-0.tif,5.598311e-08,1.391592e-04,9.609843e-01,8.997454e-10,5.023238e-18,2.668197e-19,3.755185e-10,1.467392e-13,4.182715e-03,3.469378e-02,2,HerbaceousVegetation,0


In [64]:
!cp predictions_10yr_group1_1.csv /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/satellite_gee_script/predictions_10yr_group1_1.csv

cp: cannot stat 'predictions_10yr_group1_1.csv': No such file or directory


In [65]:
df

,name,AnnualCrop,Forest,HerbaceousVegetation,Highway,Industrial,Pasture,PermanentCrop,Residential,River,SeaLake,prediction,prediction_name,imgnum
0,1082-2020-10-19-2020-12-18-6.02-1.tif,1.313343e-16,2.838180e-04,4.408744e-08,5.008692e-04,9.872841e-01,2.200774e-12,4.001908e-11,8.069928e-03,3.694312e-03,1.670730e-04,4,Industrial,1
1,2348-2021-09-29-2021-11-28-6.02-2.tif,2.274169e-15,4.698176e-05,1.482220e-04,6.446164e-05,7.573038e-01,1.075252e-11,2.385961e-11,2.217353e-05,2.403684e-01,2.045907e-03,4,Industrial,2
2,558-R-2022-03-20-2022-05-19-6.03-0.tif,5.981783e-17,1.770654e-07,2.340178e-09,1.694231e-05,9.995352e-01,1.013848e-10,1.526488e-09,4.868012e-08,4.455248e-04,2.116058e-06,4,Industrial,0
3,2732-R-2021-12-23-2022-02-21-6.03-2.tif,5.021705e-30,4.205182e-22,1.148874e-22,1.057557e-12,1.000000e+00,2.867687e-21,4.448067e-23,1.041213e-08,3.937533e-14,2.143715e-20,4,Industrial,2
4,2347-R-2021-08-15-2021-10-14-6.03-2.tif,1.556020e-24,6.285389e-15,6.293930e-15,2.895553e-08,9.999975e-01,3.720859e-15,4.249452e-15,2.293665e-06,1.739796e-07,8.653858e-13,4,Industrial,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,2737-2022-03-28-2022-05-27-6.02-0.tif,1.881614e-09,1.216938e-02,6.318500e-04,9.743893e-06,1.518973e-08,6.213490e-14,1.865554e-09,2.572356e-13,9.868587e-01,3.302993e-04,8,River,0
787,2066-2021-04-14-2021-06-13-6.02-2.tif,4.928116e-19,2.074677e-06,5.148085e-09,3.082241e-09,9.993284e-01,3.535076e-19,4.469499e-14,6.695478e-04,9.244077e-08,2.160292e-09,4,Industrial,2
788,2342-R-2020-04-30-2020-06-29-6.03-2.tif,6.058778e-26,1.261011e-13,5.190030e-16,5.973677e-12,1.000000e+00,6.380386e-17,1.566852e-18,1.729915e-08,1.150883e-11,6.406579e-17,4,Industrial,2
789,2009-2021-07-06-2021-09-04-6.02-0.tif,5.598311e-08,1.391592e-04,9.609843e-01,8.997454e-10,5.023238e-18,2.668197e-19,3.755185e-10,1.467392e-13,4.182715e-03,3.469378e-02,2,HerbaceousVegetation,0


In [66]:
final_output_cleaned = pd.merge(df, cleaned_all, how='inner', on=['name','imgnum'])

In [67]:
gdf_add = gdf[['name','imgnum','geometry']]

In [68]:
final_output_cleaned = pd.merge(final_output_cleaned, gdf_add, how='inner', on=['name','imgnum'])

In [69]:
final_output_cleaned =final_output_cleaned[['name', 'imgnum', 'label','ID','lon', 'lat', 'start_date',
       'geometry', 'AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway',
       'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River',
       'SeaLake', 'prediction', 'prediction_name']]

In [70]:
final_output_cleaned.to_csv('model_sh_attr_60_group_1_group_0.csv',index=None)

In [71]:
final_output_cleaned

,name,imgnum,label,ID,lon,lat,start_date,geometry,AnnualCrop,Forest,HerbaceousVegetation,Highway,Industrial,Pasture,PermanentCrop,Residential,River,SeaLake,prediction,prediction_name
0,558-R-2022-03-20-2022-05-19-6.03-0.tif,0,0,558,-81.965212,30.577029,2022-03-20,"POLYGON ((-81.96822 30.57398, -81.96822 30.580...",5.981783e-17,1.770654e-07,2.340178e-09,1.694231e-05,9.995352e-01,1.013848e-10,1.526488e-09,4.868012e-08,4.455248e-04,2.116058e-06,4,Industrial
1,3005-R-2022-03-15-2022-05-14-6.03-0.tif,0,0,3005,-81.740331,29.407061,2022-03-15,"POLYGON ((-81.74337 29.40402, -81.74337 29.410...",6.087063e-28,1.173694e-17,9.392016e-18,1.152249e-10,9.998141e-01,1.630646e-16,1.831100e-22,1.858800e-04,1.602782e-13,1.958756e-17,4,Industrial
2,2735-R-2022-03-15-2022-05-14-6.03-0.tif,0,0,2735,-82.812390,30.427067,2022-03-15,"POLYGON ((-82.81542 30.42405, -82.81542 30.430...",1.626214e-06,2.690494e-02,8.882167e-02,4.828038e-09,4.535216e-10,3.238060e-13,3.553606e-10,1.697191e-16,9.916592e-02,7.851058e-01,9,SeaLake
3,2462-2021-11-30-2022-01-29-6.02-0.tif,0,1,2462,-80.139901,26.212522,2021-11-30,"POLYGON ((-80.14293 26.20952, -80.14293 26.215...",5.114990e-19,1.191800e-11,5.845502e-12,1.647334e-07,1.532094e-01,3.861982e-13,1.128552e-15,8.467893e-01,1.175427e-06,1.595676e-10,7,Residential
4,3001-2021-02-17-2021-04-18-6.02-0.tif,0,1,3001,-80.210662,27.140378,2021-02-17,"POLYGON ((-80.21372 27.13730, -80.21372 27.143...",4.796299e-07,2.415345e-01,7.296392e-02,1.154025e-06,4.180694e-11,1.188996e-14,2.273527e-09,1.254593e-09,2.168085e-01,4.686915e-01,9,SeaLake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,550-2021-06-19-2021-08-18-6.02-0.tif,0,1,550,-84.324359,30.523740,2021-06-19,"POLYGON ((-84.32737 30.52071, -84.32737 30.526...",5.346881e-11,9.999671e-01,1.026444e-06,1.590784e-06,7.124859e-08,3.986413e-12,5.440245e-08,3.307553e-07,2.900378e-05,8.632931e-07,1,Forest
222,3936-R-2021-11-30-2022-01-29-6.03-0.tif,0,0,3936,-82.675134,28.839407,2021-11-30,"POLYGON ((-82.67815 28.83637, -82.67815 28.842...",8.767228e-07,3.087641e-01,4.973043e-01,1.848579e-03,4.932351e-07,2.642132e-15,1.687257e-06,2.779548e-09,1.067721e-04,1.919732e-01,2,HerbaceousVegetation
223,3286-2020-01-15-2020-03-15-6.02-0.tif,0,1,3286,-82.750478,28.137155,2020-01-15,"POLYGON ((-82.75352 28.13407, -82.75352 28.140...",3.157400e-06,5.770578e-01,2.779769e-04,3.393918e-06,9.874638e-08,2.801840e-08,1.002009e-03,4.215830e-01,1.249166e-06,7.137661e-05,1,Forest
224,2737-2022-03-28-2022-05-27-6.02-0.tif,0,1,2737,-82.115641,29.167187,2022-03-28,"POLYGON ((-82.11868 29.16417, -82.11868 29.170...",1.881614e-09,1.216938e-02,6.318500e-04,9.743893e-06,1.518973e-08,6.213490e-14,1.865554e-09,2.572356e-13,9.868587e-01,3.302993e-04,8,River


In [72]:
# df.to_csv('predictions.csv',index=None)

In [73]:
!cp predictions_10yr_group1_1.csv /content/drive/MyDrive/w210-sinkhole-risk-intelligence/4-datasets/

cp: cannot stat 'predictions_10yr_group1_1.csv': No such file or directory


In [74]:
# df['name'] = df.apply(lambda row: row['name'].replace('.jpeg','.tif'), axis = 1)

In [75]:
# final_shapefile2 = final_shapefile.reset_index(drop=True)

In [76]:
# final_shapefile2['imgnum'] = final_shapefile2.apply(lambda row: int(row['name'].split('-')[-1].replace(".tif","")),axis=1)

In [77]:
# df['imgnum'] = df.apply(lambda row: int(row['name'].split('-')[-1].replace(".tif","")),axis=1)

In [78]:
# final_shapefile2['name'] = final_shapefile2.apply(lambda row: row['name'].replace('.jpeg','.tif'), axis = 1)

In [79]:
# shapefile3 = pd.merge(final_shapefile2,df, on =['name','imgnum'],  how='inner')

In [80]:
# shapefile3.to_csv('/content/drive/MyDrive/w210/final_satellite.csv', index=None)

In [81]:
# gdf2 = gpd.GeoDataFrame(shapefile3, geometry='geometry')

In [82]:
# gdf2['start_date'] = gdf2.apply(lambda row: row['start_date'].strftime("%Y/%m/%d"),axis=1)

In [83]:
# gdf2.dtypes

In [84]:
#gdf2.to_file("/content/drive/MyDrive/w210/satellite_final_group1_365.shp")